In [1]:

#Probado con env2

from google.cloud import storage
from google.cloud import bigquery
import google.cloud
from google.oauth2 import service_account
import pandas_gbq as gbq
import pandas as pd
import datetime

In [2]:
WORKING_BUCKET = "asientos_xp_prod"
CREDENTIALS_FILENAME = "credentials.json"
GCP_PROJECT_ID = "bc-ed-advaa-dev-nc0h"

In [3]:
query = """
SELECT A.msttkt,
tktnbr,
cupon,
A.pnr_date,
concat(tktnbr, "_",A.pnr_date,"_",cupon) AS id_fila,
adv_purchase,
estadia, 
seg_fare,
trip_fare, 
ato_org_seg, 
ato_des_seg,
--IF(paid_free_awd= 'AWARD',1,0) AS award_flag,
--IF(paid_free_awd= 'PAID',1,0) AS paid_flag,
SUBSTR(nights_stay, 0, 1) AS staynight_0,
SUBSTR(nights_stay, 1, 1) AS staynight_1,
SUBSTR(nights_stay, 2, 1) AS staynight_2,
SUBSTR(nights_stay, 3, 1) AS staynight_3,
SUBSTR(nights_stay, 4, 1) AS staynight_4,
SUBSTR(nights_stay, 5, 1) AS staynight_5,
SUBSTR(nights_stay, 6, 1) AS staynight_6,
num_vlo_op ,
child_flag ,
infant_flag,
NEGOCIO,
branded_fare ,
numero_pax ,
numero_trips_res,
EXTRACT (HOUR FROM dep_datetime) AS hora_salida,
EXTRACT (MONTH FROM dep_datetime) AS mes_salida,
EXTRACT (HOUR FROM arr_datetime) AS hora_llegada,
IF(DATETIME_DIFF(arr_datetime,dep_datetime, MINUTE)>0,DATETIME_DIFF(arr_datetime,dep_datetime, MINUTE),NULL) AS duracion,
distancia,
IF(rt_flag ='RT',1,0) AS round_trip_flag,
score_corporate,
n_segmentos_lh_cliente,
numero_segmentos_cliente,
numero_reservas_cliente,
n_segmentos_business_cliente,
n_compras_asientos_cliente,
n_compras_laam_mas_cliente,
proporcion_compra_asiento_cliente,
n_canjes_cliente,
avg_fare_pnr_cliente,
avg_ap_cliente,
awarded_flag,
--avg_corp_score_cliente,
--(ancillary_fare_seat_selection) AS ancillary_fare_seat_selection,
--(ancillary_fare_latam_mas) AS ancillary_fare_latam_mas,


FROM `XP.master_asientos_predict_table` A 
LEFT JOIN `XP.OUTPUT_SEATS_EC` B
ON A.msttkt = B.msttkt
AND CAST(A.pnr_date AS STRING) = B.pnr_date 
WHERE pos='EC'
AND B.msttkt IS NULL --incremental 
AND  A.pnr_date IS NOT NULL
"""

In [6]:
credentials = service_account.Credentials.from_service_account_file(
    'service_account.json',
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

In [10]:
datos=gbq.read_gbq(query,project_id=GCP_PROJECT_ID ,dialect='standard', credentials=credentials)

In [11]:
datos

,msttkt,tktnbr,cupon,pnr_date,id_fila,adv_purchase,estadia,seg_fare,trip_fare,ato_org_seg,...,numero_segmentos_cliente,numero_reservas_cliente,n_segmentos_business_cliente,n_compras_asientos_cliente,n_compras_laam_mas_cliente,proporcion_compra_asiento_cliente,n_canjes_cliente,avg_fare_pnr_cliente,avg_ap_cliente,awarded_flag
0,4622133359811,4622133359811,4,2020-03-11,4622133359811_2020-03-11_4,185,32,11.819328,93.897993,GYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,4623384943639,4623384943639,1,2020-02-03,4623384943639_2020-02-03_1,53,26,182.500000,182.500000,GYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4622133421434,4622133421434,1,2020-09-10,4622133421434_2020-09-10_1,9,43,85.000000,85.000000,GYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4622133387250,4622133387250,1,2020-09-09,4622133387250_2020-09-09_1,100,45,16.308000,76.788000,GYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,4622133415165,4622133415165,1,2020-09-10,4622133415165_2020-09-10_1,46,94,108.250000,108.250000,GYE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,4622133426878,4622133426878,2,2020-09-10,4622133426878_2020-09-10_2,48,19,259.900000,259.900000,JFK,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,60.25417,32.800000,0
2060,4622133426878,4622133426878,1,2020-09-10,4622133426878_2020-09-10_1,48,19,208.380000,208.380000,GYE,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,60.25417,32.800000,0
2061,4622133383720,4622133383720,2,2020-09-09,4622133383720_2020-09-09_2,5,20,36.000000,36.000000,CUE,...,9.0,5.0,0.0,0.0,0.0,0.0,0.0,182.73600,4.444444,0
2062,4622133422066,4622133422066,3,2020-09-10,4622133422066_2020-09-10_3,25,21,22.158243,419.000000,UIO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
